# Prerequisites of Web Scraping
We have found an API that gives back (some) CC images geotagged in a 10KM radius around a set of coordinates. At what set of coordinates do we want to evaluate the API, so that the circles of 10KM radii cover the whole Europe? How do we make it so that we have to do the minimum possible API calls to cover Europe? That is a question we need to assess before doing any API call.

## Imports

In [1]:
import os
import sys
import math
import tqdm
import folium
import shapely
import inspect
import geopandas
import numpy as np
from pathlib import Path
from geopy import distance
import matplotlib.pyplot as plt

## Own Imports

In [2]:
sys.path.append(str(Path(os.getcwd()).parent / 'src' / 'data'))

import wikimap_api_helpers

## Preamble
I found out that the optimal covering of an (infinite) area using circles of radii $r$ is by arranging the circles in a honeycomb structure. That means, in rows of circles, with each row having an intendent alternatingly. Alternatively, this means that the centres of every 3 adjacent circles are form an equilateral triangle. This optimal arrangement is reached when the horizontal spacing is $r\sqrt{3}$, with an alternating intendation of $\frac{r\sqrt{3}}{2}$, and a vertical distance of $\frac{3r}{2}$. The following cell is just a sanity test.

In [ ]:
r = 10000
figure, axes = plt.subplots()
Drawing_uncolored_circle1 = plt.Circle( (0, 0 ),
                                      r,
                                      fill = False )
Drawing_uncolored_circle2 = plt.Circle( (math.sqrt(3)*r, 0 ),
                                      r,
                                      fill = False )
Drawing_uncolored_circle3 = plt.Circle( (math.sqrt(3)*r/2, r*3/2 ),
                                      r,
                                      fill = False )
Drawing_uncolored_circle4 = plt.Circle( (math.sqrt(3)*r*2, 0 ),
                                      r,
                                      fill = False )
Drawing_uncolored_circle5 = plt.Circle( (math.sqrt(3)*r/2 + math.sqrt(3)*r, r*3/2 ),
                                      r,
                                      fill = False )

 
axes.set_aspect(1 )
axes.add_artist(Drawing_uncolored_circle1)
axes.add_artist(Drawing_uncolored_circle2)
axes.add_artist(Drawing_uncolored_circle3)
axes.add_artist(Drawing_uncolored_circle4)
axes.add_artist(Drawing_uncolored_circle5)
axes.set_xlim(-1.2*r, 5*r)
axes.set_ylim(-1.2*r, 3*r)
plt.title( 'Circles in Beehive Formation' )
plt.show()

## Fetching the countries' geoshapes
Now we load the geoshapes of the European countries. The neat thing is that, given a coordinate pair, the geoshape has a method to check whether the point is within its borders. Very useful!

In [ ]:
shapefile_path = Path(os.getcwd()).parent / 'data' / 'external' / 'geoshapes' / 'country_geoshapes.shx'
geo_df = geopandas.read_file(shapefile_path)

In [ ]:
geo_df.query('LEVL_CODE == 0').head() # quick check

In [ ]:
geo_df.loc[34, 'geometry'] # how does the Netherlands look like?

### Small test
Here we check whether the ```shape.contains(point)``` method is reliable and useful for our purposes.

In [ ]:
cr = geo_df.loc[1, 'geometry']

x = []
y = []
mask = []
for i in tqdm.tqdm(np.arange(cr.bounds[0], cr.bounds[2], 0.01)):
    for j in np.arange(cr.bounds[1], cr.bounds[3], 0.01):
        x.append(i)
        y.append(j)
        mask.append(geo_df.loc[1, 'geometry'].contains(shapely.geometry.Point(i, j)))

In [ ]:
plt.scatter(x, y, c = mask) # we colour a point yellow if is inside the shape, purple if outside
plt.show() # that's a very accurate representation of the Czech Republic!

## Algorithms to generate the grid of points
There are two ways of doing this: either we call the API in points that are in the country, OR we call the API in points such that the 10KM radius circle and the country have a nonempty intersection. The first way is simple, the second requires a bit of thought, and a function to create a circle from a point. That is the function ```generate_circle(point)```.

In [ ]:
inspect.getsourcelines(wikimap_api_helpers.generate_circle)

In [ ]:
i = 0
for lat, lon in zip(y, x):
    # Here we see if the circles have the correct size.
    # We print the distance from centre to side, and from centre to top.
    point = shapely.geometry.Point(lon, lat)
    print(
        distance.geodesic(
        (point.bounds[1], point.bounds[0]),
        (wikimap_api_helpers.generate_circle(point).bounds[1], point.bounds[0])).m,

        distance.geodesic(
        (point.bounds[1], point.bounds[0]),
        (point.bounds[1], wikimap_api_helpers.generate_circle(point).bounds[0])).m
    )
    if i > 10:
        break
    i += 1

The following procedure generates the grid of points where we should call the API. How it works: it begins at the southernmost, westmost point of the box that encloses the country. Then takes 10KM steps towards the east. At each step evaluates if the point is in the country (alternatively, if its corresponding circle intersects the country). When the procedure steps too far away into the east, out of the box that contains the country, then it returns to the westernmost point, but 10KM to the north of it. Then begins the walk eastwards again. This is repeated until the procedure oversteps the northernmost, eastmost corner of the box.

In [ ]:
inspect.getsourcelines(wikimap_api_helpers.get_query_points)

In [ ]:
lats, lons, mask = wikimap_api_helpers.get_query_points(shape = geo_df.loc[34, 'geometry'])

In [ ]:
print('Number of points needed to scrape the Netherlands: ', np.array(mask).sum())

In [ ]:
plt.scatter(lons, lats, c = mask) # see how it looks! same idea as before
plt.show() # we do not need that many points!

## Quick visualization
Notice that this plot is just an approximation: the radii of the circles is just 26-pixels long. If you zoom in or out, their size changes. So this map oly checks that the distances between points are moreless accurate.

In [ ]:
ne_shape = geo_df.loc[34, 'geometry']
ne_map = folium.Map([ne_shape.centroid.bounds[1], ne_shape.centroid.bounds[0], ], zoom_start=8)
for lat, lon, is_in in zip(lats, lons, mask):
    if is_in:
        folium.vector_layers.CircleMarker(
            location=[lat, lon],
            radius=26,
            color='#3186cc',
            fill=False    
        ).add_to(ne_map)

ne_map # ATTENTION: circle radii are approximated

## API call code

In [ ]:
inspect.getsourcelines(wikimap_api_helpers.query_at)

## Some more circles...
... in order to understand why, when an API call returns too many results, we do 4 calls at 4 points around the original API coordinates, with a radius that is $r_{new} = \frac{r_{old}}{2}$. If we took instead 4 circles of radius $r_{new} = \frac{r_{old}}{\sqrt{2}}$ (which in a sense would be ideal because it would cover the whole area) we would overstep outside of the original area, which would make the API calling very long. After all $\frac{r_{old}}{\sqrt{2}} + \frac{r_{old}}{\sqrt{2}^2} + \frac{r_{old}}{\sqrt{2}^3} + \frac{r_{old}}{\sqrt{2}^4} + ... = \frac{r_{old}}{1 - \frac{1}{\sqrt{2}}} = 3.4142*r_{old}$. So we would be over three radii away of the centre!

In [ ]:
### if we divide radius by two
r = 1
nr = r/2
figure, axes = plt.subplots()
Drawing_uncolored_circle0 = plt.Circle( (0, 0 ),
                                      r,
                                      fill = False )
Drawing_uncolored_circle1 = plt.Circle( (nr/math.sqrt(2), nr/math.sqrt(2)) ,
                                      nr,
                                      fill = False )
Drawing_uncolored_circle2 = plt.Circle( (nr/math.sqrt(2), -nr/math.sqrt(2)) ,
                                      nr,
                                      fill = False )
Drawing_uncolored_circle3 = plt.Circle( (-nr/math.sqrt(2), nr/math.sqrt(2)) ,
                                      nr,
                                      fill = False )
Drawing_uncolored_circle4 = plt.Circle( (-nr/math.sqrt(2), -nr/math.sqrt(2)) ,
                                      nr,
                                      fill = False )

 
axes.set_aspect(1 )
axes.add_artist(Drawing_uncolored_circle0)
axes.add_artist(Drawing_uncolored_circle1)
axes.add_artist(Drawing_uncolored_circle2)
axes.add_artist(Drawing_uncolored_circle3)
axes.add_artist(Drawing_uncolored_circle4)
axes.set_xlim(-2*r, 2*r)
axes.set_ylim(-2*r, 2*r)
plt.title( 'Circle, with smaller circles of radius r/2' )
plt.show()

In [ ]:
# if we divide radius by sqrt(2)
r = 1
nr = r/math.sqrt(2)
figure, axes = plt.subplots()
Drawing_uncolored_circle0 = plt.Circle( (0, 0 ),
                                      r,
                                      fill = False )
Drawing_uncolored_circle1 = plt.Circle( (nr/math.sqrt(2), nr/math.sqrt(2)) ,
                                      nr,
                                      fill = False )
Drawing_uncolored_circle2 = plt.Circle( (nr/math.sqrt(2), -nr/math.sqrt(2)) ,
                                      nr,
                                      fill = False )
Drawing_uncolored_circle3 = plt.Circle( (-nr/math.sqrt(2), nr/math.sqrt(2)) ,
                                      nr,
                                      fill = False )
Drawing_uncolored_circle4 = plt.Circle( (-nr/math.sqrt(2), -nr/math.sqrt(2)) ,
                                      nr,
                                      fill = False )

 
axes.set_aspect(1)
axes.add_artist(Drawing_uncolored_circle0)
axes.add_artist(Drawing_uncolored_circle1)
axes.add_artist(Drawing_uncolored_circle2)
axes.add_artist(Drawing_uncolored_circle3)
axes.add_artist(Drawing_uncolored_circle4)
axes.set_xlim(-2*r, 2*r)
axes.set_ylim(-2*r, 2*r)
plt.title( 'Circle, with smaller circles of radius sqrt(r)' )
plt.show()